In [1]:
import numpy as np
import tensorly as tl

Using numpy backend.


# Defining our example tensor

In [3]:
X = tl.tensor(np.arange(24).reshape((3, 4, 2)))

In [4]:
X

array([[[  0.,   1.],
        [  2.,   3.],
        [  4.,   5.],
        [  6.,   7.]],

       [[  8.,   9.],
        [ 10.,  11.],
        [ 12.,  13.],
        [ 14.,  15.]],

       [[ 16.,  17.],
        [ 18.,  19.],
        [ 20.,  21.],
        [ 22.,  23.]]])

You can view the frontal slices by fixing the last axis:

In [5]:
X[:, :, 0]

array([[  0.,   2.,   4.,   6.],
       [  8.,  10.,  12.,  14.],
       [ 16.,  18.,  20.,  22.]])

In [6]:
X[:, :, 1]

array([[  1.,   3.,   5.,   7.],
       [  9.,  11.,  13.,  15.],
       [ 17.,  19.,  21.,  23.]])

# 2.Setting the backend

In TensorLy you can dynamically set the backend to use either NumPy or PyTorch to represent tensors and perform the operations:

In [7]:
type(X)

numpy.ndarray

By default, the backend is set to NumPy, here is how to change it, first to PyTorch:

In [12]:
tl.set_backend('pytorch')

Using pytorch backend.


In [15]:
X = tl.tensor(np.arange(24).reshape((3, 4, 2)), device='cpu')
type(X)

torch.Tensor

In [16]:
tl.context(X)

{'device': device(type='cpu'), 'dtype': torch.float32, 'requires_grad': False}

As expected tensors are now represented as a PyTorch tensor. 

Let's change it back to NumPy for the rest of the tutorial.

In [10]:
tl.set_backend('numpy')

Using numpy backend.


In [11]:
X = tl.tensor(np.arange(24).reshape((3, 4, 2)))
type(X)

numpy.ndarray

# 3.Basic tensor operations

## 3.1 Unfolding

Also called **matrization**, **unfolding** a tensor is done by reading the element in a given way as to obtain a matrix instead of a tensor.

It is done by stacking the **fibers** of the tensor into a matrix.

![tensor_illustration](images/example-unfolding-fibers.png)


### Convention

   Remember that, to be consistent with the Python indexing that always starts at zero,
   in tensorly, modes (and therefore unfolding) also start at zero!

   Therefore ``unfold(tensor, 0)`` will unfold said tensor along its first dimension!
   


In [11]:
tl.unfold(X, mode=0)

array([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.],
       [ 16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.]])

In [12]:
tl.unfold(X, mode=1)

array([[  0.,   1.,   8.,   9.,  16.,  17.],
       [  2.,   3.,  10.,  11.,  18.,  19.],
       [  4.,   5.,  12.,  13.,  20.,  21.],
       [  6.,   7.,  14.,  15.,  22.,  23.]])

In [13]:
tl.unfold(X, mode=2)

array([[  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,
         22.],
       [  1.,   3.,   5.,   7.,   9.,  11.,  13.,  15.,  17.,  19.,  21.,
         23.]])

## 3.2 Folding

Folding is the inverse operation: you can **fold** an unfolded tensor back from matrix to full tensor using the ``tensorly.fold`` function.

In [14]:
unfolding = tl.unfold(X, 1)
original_shape = X.shape
tl.fold(unfolding, mode=1, shape=original_shape)

array([[[  0.,   1.],
        [  2.,   3.],
        [  4.,   5.],
        [  6.,   7.]],

       [[  8.,   9.],
        [ 10.,  11.],
        [ 12.,  13.],
        [ 14.,  15.]],

       [[ 16.,  17.],
        [ 18.,  19.],
        [ 20.,  21.],
        [ 22.,  23.]]])

## 3.3 n-mode product

Also known as **tensor contraction**. This is a natural generalization of matrix-vector and matrix-matrix product. When multiplying a tensor by a matrix or a vector, we now have to specify the **mode** $n$ along which to take the product.
### Tensor times matrix

In that case we are doing an operation analogous to a matrix multiplication on the $n$-th mode. Given a tensor $\tilde X$ of size $(I_0, I_1, \cdots, I_N)$, and a matrix $M$ of size $(D, I_n)$, the $n$-mode product of $\tilde X$ by $M$ is written $\tilde X \times_n M$ and is of size $(I_k, I_0 \times \cdots \times I_{n-1} \times D \times I_{n+1} \cdots \times I_n)$.

### Tensor times vector

In that case we are contracting over the $n$-th mode by multiplying it with a vector. Given a tensor $\tilde X$ of size $(I_0, I_1, \cdots, I_N)$, and a vector $v$ of size $(I_n)$, the $n$-mode product of $\tilde X$ by $v$ is written $\tilde X \times_n v$ and is of size $(I_k, I_0 \times \cdots \times I_{n-1} \times I_{n+1} \cdots \times I_n)$.


### Example

In TensorLy, all the tensor algebra functions are located in the `tensorly.tenalg` module. For the n-mode product, you will need to use the function `mode_dot` that works transparently for multiplying a tensor by a matrix or a vector along a given mode.

#### Tensor times matrix

With the tensor $\tilde X$ of size (3, 4, 2) we defined previously, let's define a matrix M of size (5, 4) to multiply along the second mode:

In [15]:
M = tl.tensor(np.arange(4*5).reshape((5, 4)))
print(M.shape)

(5, 4)


Keep in mind indexing starts at zero, so the second mode is represented by `mode=1`:

In [16]:
res = tl.tenalg.mode_dot(X, M, mode=1)

As expected the result is of shape (3, 5, 2)

In [17]:
res.shape

(3, 5, 2)

#### Tensor times vector

Similarly, we can contract along the mode 1 with a vector of size 4 (our tensor is of size (3, 4, 2).


In [18]:
v = tl.tensor(np.arange(4))
print(v.shape)

(4,)


In [19]:
res = tl.tenalg.mode_dot(X, v, mode=1)

Since we have multiplied by a vector, we have effectively contracted out one mode of the tensor so the result is a matrix:

In [20]:
res.shape

(3, 2)

## Kronecker and Khatri-Rao product

In [21]:
from tensorly.tenalg import kronecker, khatri_rao

In [22]:
A = tl.tensor([[2, 1],
               [3, 4]])

In [23]:
B = tl.tensor([[0.5, 1],
               [2, 0]])

The Kronecker and Khatri-Rao product take as input a list of matrices (as they can take the kronecker and khatri-rao product of more than one matrix)

In [24]:
kronecker([A, B])

array([[ 1. ,  2. ,  0.5,  1. ],
       [ 4. ,  0. ,  2. ,  0. ],
       [ 1.5,  3. ,  2. ,  4. ],
       [ 6. ,  0. ,  8. ,  0. ]])

In [25]:
khatri_rao([A, B])

array([[ 1. ,  1. ],
       [ 4. ,  0. ],
       [ 1.5,  4. ],
       [ 6. ,  0. ]])

Compare that to the result shown in the slides :)